In [ ]:
# Import tdmclient Notebook environment:
import tdmclient.notebook
from Kalman import ekf
await tdmclient.notebook.start()

In [ ]:

@tdmclient.notebook.sync_var 
def set_speed(right_speed,left_speed):
    global motor_right_target, motor_left_target
    motor_right_target=right_speed
    motor_left_target=left_speed

@tdmclient.notebook.sync_var
def stop_thymio():
    global motor_right_target,motor_left_target
    motor_right_target=0
    motor_left_target=0

@tdmclient.notebook.sync_var
def get_speed():
    global motor_right_speed, motor_left_speed
    return motor_right_speed,motor_left_speed


In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def speed_convesion(r_speed,l_speed):
    thymio_speed_to_mms = 0.430 # value found in covariance_estimation

    #odometry 
    avg_thymio_speed = (r_speed + l_speed) / 2
    speed = avg_thymio_speed * thymio_speed_to_mms # [mm/s]
    return speed

def angular_vel_conversion(r_speed,l_speed):
    d = 95 # distance between the 2 wheels [mm]
    thymio_speed_to_mms = 0.430 # value found in covariance_estimation
    
    difference_speed = l_speed - r_speed
    omega = difference_speed * thymio_speed_to_mms / d # [rad/s]

    return omega
print()

In [ ]:


state_est_prev=[0,0,0,0,0]
P_prev=np.diag([0.1,0.1,0.1,0.1,0.1])

obs_camera=[np.nan,np.nan,np.nan]

camera_state=False


# faire une boucle ici
actual_v=speed_convesion(get_speed())
actual_w=angular_vel_conversion(get_speed())
obs_odometry=np.concatenate((actual_v,actual_w))

input_v=speed_convesion(r_speed,l_speed)
input_w=angular_vel_conversion(r_speed,l_speed)
control_vect_prev=np.concatenate((input_v,input_w))


state_est, P_est = ekf(state_est_prev, control_vect_prev, P_prev, obs_camera, obs_odometry, camera_state)

state_est_prev=state_est
P_prev=P_est